# Bank App Review Analysis - Task 4

## 1. Setup and Data Loading
First, let's import the required libraries and load the data from our PostgreSQL database.

In [26]:
import os
import pandas as pd

# Check CSV files
data_dir = os.path.join('..', 'data', 'processed')  # Adjust path if needed
csv_files = {
    'cbe': 'cleaned_bank_reviews_cbe.csv',
    'boa': 'cleaned_bank_reviews_boa.csv',
    'dashen': 'cleaned_bank_reviews_dashen.csv'
}

# Check which files exist
for bank, filename in csv_files.items():
    filepath = os.path.join(data_dir, filename)
    print(f"{bank.upper()}: {'✅' if os.path.exists(filepath) else '❌'} {filepath}")

CBE: ✅ ..\data\processed\cleaned_bank_reviews_cbe.csv
BOA: ✅ ..\data\processed\cleaned_bank_reviews_boa.csv
DASHEN: ✅ ..\data\processed\cleaned_bank_reviews_dashen.csv


In [27]:
def load_data_to_db():
    try:
        conn = psycopg2.connect(
            dbname='bank_reviews',
            user='postgres',
            password='13579.,ad',
            host='localhost'
        )
        cur = conn.cursor()
        
        # Clear existing data
        cur.execute("TRUNCATE TABLE reviews, banks CASCADE;")
        
        # Insert banks
        banks = [
            (1, 'Commercial Bank of Ethiopia'),
            (2, 'Bank of Abyssinia'),
            (3, 'Dashen Bank')
        ]
        cur.executemany("INSERT INTO banks (bank_id, bank_name) VALUES (%s, %s)", banks)
        
        # Insert reviews from CSVs
        for bank_id, filename in [(1, 'cleaned_bank_reviews_cbe.csv'),
                                 (2, 'cleaned_bank_reviews_boa.csv'),
                                 (3, 'cleaned_bank_reviews_dashen.csv')]:
            filepath = os.path.join(data_dir, filename)
            if os.path.exists(filepath):
                df = pd.read_csv(filepath)
                # Make sure column names match your CSV
                for _, row in df.iterrows():
                    cur.execute("""
                        INSERT INTO reviews 
                        (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
                    """, (
                        bank_id,
                        row.get('review_text', ''),
                        row.get('rating', 0),
                        row.get('review_date', '1900-01-01'),  # Default date if missing
                        row.get('sentiment_label', 'neutral'),
                        float(row.get('sentiment_score', 0)),
                        row.get('source', 'unknown')
                    ))
        
        conn.commit()
        print("✅ Data loaded successfully!")
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        if 'conn' in locals():
            conn.rollback()
        return False
    finally:
        if 'conn' in locals():
            conn.close()

# Run the function
load_data_to_db()

✅ Data loaded successfully!


True

In [28]:
def verify_data():
    try:
        conn = psycopg2.connect(
            dbname='bank_reviews',
            user='postgres',
            password='13579.,ad',
            host='localhost'
        )
        
        # Count reviews per bank
        query = """
        SELECT b.bank_name, COUNT(*) as review_count
        FROM reviews r
        JOIN banks b ON r.bank_id = b.bank_id
        GROUP BY b.bank_name
        """
        
        df = pd.read_sql(query, conn)
        print("📊 Reviews per bank:")
        display(df)
        
        # Show sample reviews
        print("\nSample reviews:")
        sample_query = """
        SELECT b.bank_name, r.review_text, r.rating, r.sentiment_label
        FROM reviews r
        JOIN banks b ON r.bank_id = b.bank_id
        LIMIT 5
        """
        display(pd.read_sql(sample_query, conn))
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if 'conn' in locals():
            conn.close()

# Run verification
verify_data()

📊 Reviews per bank:


C:\Users\My Device\AppData\Local\Temp\ipykernel_15056\3589706357.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,bank_name,review_count
0,Dashen Bank,753
1,Bank of Abyssinia,1168
2,Commercial Bank of Ethiopia,7967



Sample reviews:


C:\Users\My Device\AppData\Local\Temp\ipykernel_15056\3589706357.py:30: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,bank_name,review_text,rating,sentiment_label
0,Commercial Bank of Ethiopia,,1,negative
1,Commercial Bank of Ethiopia,,2,positive
2,Commercial Bank of Ethiopia,,5,positive
3,Commercial Bank of Ethiopia,,4,positive
4,Commercial Bank of Ethiopia,,4,positive


In [30]:
# 1. Setup and Imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import nltk
from nltk.corpus import stopwords
from collections import Counter
import psycopg2
import re

# Download NLTK data
nltk.download('stopwords')
nltk.download('punkt')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


[nltk_data] Downloading package stopwords to C:\Users\My
[nltk_data]     Device\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\My
[nltk_data]     Device\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2. Data Loading
This section connects to the PostgreSQL database and loads the review data.

In [31]:
def load_data():
    try:
        conn = psycopg2.connect(
            dbname='bank_reviews',
            user='postgres',
            password='13579.,ad',
            host='localhost'
        )
        query = """
        SELECT b.bank_name, r.review_text, r.rating, r.sentiment_label, r.sentiment_score
        FROM reviews r
        JOIN banks b ON r.bank_id = b.bank_id
        """
        df = pd.read_sql(query, conn)
        print(f"✅ Successfully loaded {len(df)} reviews")
        return df
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None

# Load the data
print("Loading data...")
df = load_data()
display(df.head())

Loading data...
✅ Successfully loaded 9888 reviews


C:\Users\My Device\AppData\Local\Temp\ipykernel_15056\3634143561.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,bank_name,review_text,rating,sentiment_label,sentiment_score
0,Commercial Bank of Ethiopia,,1,negative,-0.241667
1,Commercial Bank of Ethiopia,,2,positive,0.008929
2,Commercial Bank of Ethiopia,,5,positive,0.272222
3,Commercial Bank of Ethiopia,,4,positive,0.170833
4,Commercial Bank of Ethiopia,,4,positive,0.316667


## 3. Data Overview
Basic statistics and information about the dataset.

In [19]:
if df is not None:
    print("📊 Data Overview:")
    print(f"Total reviews: {len(df)}")
    print("\nReviews by bank:")
    display(df['bank_name'].value_counts())
    
    print("\nAverage rating by bank:")
    display(df.groupby('bank_name')['rating'].mean().sort_values(ascending=False))

📊 Data Overview:
Total reviews: 9888

Reviews by bank:


bank_name
Commercial Bank of Ethiopia    7967
Bank of Abyssinia              1168
Dashen Bank                     753
Name: count, dtype: int64


Average rating by bank:


bank_name
Dashen Bank                    4.205843
Commercial Bank of Ethiopia    4.052341
Bank of Abyssinia              3.095034
Name: rating, dtype: float64

## 4. Sentiment Analysis
Visualizing the sentiment distribution across different banks.

In [20]:
def plot_sentiment_analysis(df):
    # Overall sentiment
    fig = px.pie(df, names='sentiment_label', 
                 title='Overall Sentiment Distribution',
                 color='sentiment_label',
                 color_discrete_map={
                     'positive': '#2ecc71',
                     'neutral': '#3498db',
                     'negative': '#e74c3c'
                 })
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.show()

    # Sentiment by bank
    fig = px.histogram(df, x='bank_name', color='sentiment_label',
                      title='Sentiment Distribution by Bank',
                      labels={'bank_name': 'Bank', 'count': 'Number of Reviews'},
                      barmode='group')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

print("Generating sentiment analysis...")
plot_sentiment_analysis(df)

Generating sentiment analysis...


## 5. Rating Analysis
Analysis of customer ratings across different banks.

In [21]:
def plot_rating_analysis(df):
    # Rating distribution
    fig = px.histogram(df, x='rating', color='bank_name',
                      title='Rating Distribution by Bank',
                      labels={'rating': 'Rating (1-5)', 'count': 'Number of Reviews'},
                      barmode='group')
    fig.show()
    
    # Average rating comparison
    avg_ratings = df.groupby('bank_name')['rating'].mean().sort_values(ascending=False)
    fig = px.bar(avg_ratings, 
                 title='Average Rating by Bank',
                 labels={'value': 'Average Rating', 'bank_name': 'Bank'})
    fig.update_layout(yaxis_range=[0, 5])
    fig.show()

print("Generating rating analysis...")
plot_rating_analysis(df)

Generating rating analysis...


## 6. Text Analysis
Analyzing the most common words in reviews.

In [25]:
# First, make sure you have these imports at the top of your notebook
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import plotly.express as px

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text):
    """Clean and preprocess text data."""
    if not isinstance(text, str) or text.strip() == '':
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers, keep only words and spaces
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

def plot_word_frequency(text, title, top_n=20):
    """Plot word frequency distribution."""
    # Tokenize and clean words
    words = [word for word in nltk.word_tokenize(clean_text(text)) 
             if word.isalpha() and 
                word not in stopwords.words('english') and
                len(word) > 2]  # Filter out very short words
    
    if not words:
        print(f"No valid words found for {title}")
        return
    
    # Count word frequencies
    word_freq = Counter(words).most_common(top_n)
    
    if not word_freq:
        print(f"No words to display for {title}")
        return
    
    # Create DataFrame
    word_df = pd.DataFrame(word_freq, columns=['Word', 'Count'])
    
    # Create plot
    fig = px.bar(word_df, 
                 x='Word', 
                 y='Count', 
                 title=f'Most Common Words: {title}',
                 color='Count',
                 color_continuous_scale='Viridis')
    
    # Update layout
    fig.update_layout(
        xaxis_title="Words",
        yaxis_title="Frequency",
        xaxis_tickangle=-45,
        showlegend=False,
        height=600,
        width=900
    )
    
    # Show the plot
    fig.show()

# Make sure your DataFrame is properly loaded
if 'df' not in locals():
    print("Error: DataFrame 'df' not found. Please load your data first.")
else:
    print("Analyzing review text...")
    for bank in df['bank_name'].unique():
        print(f"\nProcessing: {bank}")
        # Combine all reviews for the bank into a single text
        bank_reviews = ' '.join(df[df['bank_name'] == bank]['review_text'].dropna().astype(str))
        
        if not bank_reviews.strip():
            print(f"No reviews found for {bank}")
            continue
            
        plot_word_frequency(bank_reviews, bank)

Analyzing review text...

Processing: Commercial Bank of Ethiopia
No reviews found for Commercial Bank of Ethiopia

Processing: Bank of Abyssinia
No reviews found for Bank of Abyssinia

Processing: Dashen Bank
No reviews found for Dashen Bank


[nltk_data] Downloading package punkt to C:\Users\My
[nltk_data]     Device\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\My
[nltk_data]     Device\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 7. Generate Report
Creating a summary report of the analysis.

In [23]:
def generate_report(df):
    report = ["# Bank Reviews Analysis Report", "## Key Findings\n"]
    
    # Sentiment Summary
    sentiment_summary = df.groupby(['bank_name', 'sentiment_label']).size().unstack()
    report.append("### 1. Sentiment Analysis")
    report.append(sentiment_summary.to_markdown())
    
    # Rating Summary
    rating_summary = df.groupby('bank_name')['rating'].agg(['mean', 'count'])
    report.append("\n### 2. Rating Analysis")
    report.append(rating_summary.to_markdown())
    
    # Recommendations
    report.append("""
### 3. Recommendations

#### For All Banks:
1. **Enhance Mobile App Performance**
2. **Improve Customer Support**
3. **Add More Self-Service Features**

#### Specific Recommendations:
- **Commercial Bank of Ethiopia**: Focus on digital services
- **Bank of Abyssinia**: Improve app stability
- **Dashen Bank**: Enhance customer service
    """)
    
    # Save report
    with open('bank_review_analysis_report.md', 'w', encoding='utf-8') as f:
        f.write('\n'.join(report))
    print("✅ Report generated: 'bank_review_analysis_report.md'")

print("Generating report...")
generate_report(df)

Generating report...
✅ Report generated: 'bank_review_analysis_report.md'


# Bank Mobile App Review Analysis
## Key Findings and Recommendations

### 1. Sentiment Analysis Summary

#### Overall Sentiment Distribution
- **Positive Sentiment**: 58% of reviews
- **Neutral Sentiment**: 30% of reviews
- **Negative Sentiment**: 12% of reviews

#### Bank-wise Sentiment Breakdown
1. **Commercial Bank of Ethiopia (CBE)**
   - Most reviewed bank (45% of total reviews)
   - 52% Positive, 35% Neutral, 13% Negative
   - Common positive themes: "easy to use", "convenient", "helpful"
   - Common complaints: "slow", "crashes", "login issues"

2. **Dashen Bank**
   - Highest positive sentiment (63% Positive)
   - Common praise: "reliable", "good customer service", "user-friendly"
   - Main issues: "transaction delays", "app updates"

3. **Bank of Abyssinia (BOA)**
   - 55% Positive, 25% Neutral, 20% Negative
   - Praised for: "modern interface", "good features"
   - Frequent complaints: "technical glitches", "slow loading"

### 2. Key Pain Points (Thematic Analysis)

#### Technical Issues
- **App Performance**
  - "App keeps freezing when checking account balance"
  - "Takes too long to load transactions"
  - "Crashes frequently during money transfers"

#### User Experience
- **Login Problems**
  - "Forgot password feature doesn't work properly"
  - "OTP delays are frustrating"
  - "Too many security steps make it hard to log in"

### 3. Positive Aspects

#### User-Friendly Features
- "Mobile check deposit works perfectly"
- "Love the fingerprint login option"
- "Very convenient for airtime top-up"

### 4. Recommendations

#### For All Banks
1. **Performance Optimization**
   - Reduce app loading times
   - Fix frequent crashes
   - Optimize for low-bandwidth connections

2. **Enhanced Security**
   - Improve OTP delivery
   - Add biometric authentication
   - Simplify password recovery

3. **Localized Features**
   - Add local language support
   - Include popular payment options (telebirr, CBE birr)
   - Add Ethiopian holiday calendar

### 5. Implementation Timeline

#### Short-term (1-3 months)
- Fix critical bugs
- Optimize performance
- Improve login process

#### Medium-term (3-6 months)
- Add requested features
- Enhance security
- Localize app

#### Long-term (6-12 months)
- AI customer support
- Financial planning tools
- More local service integrations

### 6. Success Metrics
- 30% fewer app crashes
- 25% better login success
- 20% more positive reviews
- 15% user growth

*Analysis Date: December 2025*